In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

from pathlib import Path
from os import chdir

In [3]:
# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [4]:
# cost optimal solution
res_basic = Results("../data/outputs/03_example_CCTS")

In [5]:
# show the deafult analysis settings
pprint(res_basic.results[None]["analysis"])

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/extensions/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins.extensions')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').decla

{'dataset': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS',
 'earliest_year_of_data': 1900,
 'folder_output': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/outputs',
 'header_data_inputs': {'set_capacity_types': 'capacity_type',
                        'set_carriers': 'carrier',
                        'set_conversion_technologies': 'technology',
                        'set_dependent_carriers': 'carrier',
                        'set_edges': 'edge',
                        'set_elements': 'element',
                        'set_failure_states': 'failure_states',
                        'set_input_carriers': 'carrier',
                        'set_lca_impact_categories': 'impact_category',
                        'set_location': 'location',
                        'set_nodes': 'node',
                        'set_output_carriers': 'carrier',
                        'set_retrofitting_technologies': 'technology',
                        'set_stor

In [6]:
# get capacity addition and save as csv
res_basic.get_df('capacity_addition', to_csv="capacity_addition")

technology     capacity_type  location               year
capture_amine  power          swiss_storage_CH       0       0.0
                                                     1       0.0
                                                     2       0.0
                                                     3       0.0
                                                     4       0.0
                                                            ... 
truck_batch    power          waste_2_CH-waste_1_CH  0       0.0
                                                     1       0.0
                                                     2       0.0
                                                     3       0.0
                                                     4       0.0
Name: capacity_addition, Length: 170, dtype: float64

In [7]:
# get capacity csv
df = pd.read_csv('capacity_addition.csv')

In [8]:
df

,technology,capacity_type,location,year,capacity_addition
0,capture_amine,power,swiss_storage_CH,0,0.0
1,capture_amine,power,swiss_storage_CH,1,0.0
2,capture_amine,power,swiss_storage_CH,2,0.0
3,capture_amine,power,swiss_storage_CH,3,0.0
4,capture_amine,power,swiss_storage_CH,4,0.0
...,...,...,...,...,...
165,truck_batch,power,waste_2_CH-waste_1_CH,0,0.0
166,truck_batch,power,waste_2_CH-waste_1_CH,1,0.0
167,truck_batch,power,waste_2_CH-waste_1_CH,2,0.0
168,truck_batch,power,waste_2_CH-waste_1_CH,3,0.0


In [9]:
# find unique technologies
technology = df['technology'].unique()
technology

array(['capture_amine', 'compression_16_110bar', 'condition_110bar',
       'condition_16bar', 'decompression_110_16bar',
       'decompression_110bar_ambient', 'emergency_storage', 'emitter_wte',
       'heat_pump', 'permanent_storage', 'truck_batch'], dtype=object)

In [11]:
# create existing capacity files

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS_N-1')
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/capacity_existing.csv')
    
    #Clean up table
    x = x.drop(columns='capacity_type')
    x = x.drop(columns='technology')
    x = x[x['capacity_addition'] != 0]
    x = x.rename(columns={'year': 'year_construction'})
    x = x.rename(columns={'capacity_addition': 'capacity_existing'})
    x['year_construction'] = x['year_construction'] + 2025 
    
    if  tec == 'biomethane_boiler':
        x['capacity_existing'] = x['capacity_existing'] / 1000
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)